In [14]:
import os
import boto3
import importlib
from botocore.config import Config
from datetime import datetime, timedelta

In [15]:
import functions
importlib.reload(functions)
from functions import *

In [2]:
session = boto3.Session(
    aws_access_key_id="1c407b86-9d98-4a01-bf59-3b45eed92548",
    aws_secret_access_key="EuwKhItrKdGMxjTLTrltOOX5aJQ0ozpp",
)

In [3]:
s3 = session.client(
    "s3",
    endpoint_url="https://files.polygon.io",
    config=Config(signature_version="s3v4"),
)

In [4]:
paginator = s3.get_paginator("list_objects_v2")

In [25]:
prefix = "us_stocks_sip/day_aggs_v1"
bucket = "flatfiles"

In [48]:
allfilenames = []

for page in paginator.paginate(Bucket=bucket, prefix=prefix):
    for obj in page["Contents"]:
        allfilenames.append(obj["Key"])

In [49]:
allfilenames[0]

'global_crypto/day_aggs_v1/2013/11/2013-11-04.csv.gz'

In [52]:
prefixes = [item.split("/")[0] +'/'+ item.split('/')[1] for item in allfilenames]

In [53]:
set(prefixes)

{'global_crypto/day_aggs_v1',
 'global_crypto/minute_aggs_v1',
 'global_crypto/trades_v1',
 'global_forex/day_aggs_v1',
 'global_forex/minute_aggs_v1',
 'global_forex/quotes_v1',
 'us_indices/day_aggs_v1',
 'us_indices/minute_aggs_v1',
 'us_indices/values_v1',
 'us_options_opra/day_aggs_v1',
 'us_options_opra/minute_aggs_v1',
 'us_options_opra/quotes_v1',
 'us_options_opra/trades_v1',
 'us_stocks_sip/day_aggs_v1',
 'us_stocks_sip/minute_aggs_v1',
 'us_stocks_sip/quotes_v1',
 'us_stocks_sip/trades_v1'}

In [54]:
dag = [item for item in allfilenames if item.startswith('us_stocks_sip/day_aggs_v1')]

In [8]:
dayaggs = os.listdir('day_aggs')

In [33]:
havedates = [datefromfilename(x) for x in dayaggs]
missingdates(havedates, availabledates)

In [ ]:
downloadable = [x for x in allfilenames if datefromfilename(x) >= datefromfilename(dayaggs[0])]
availabledates = [datefromfilename(x) for x in downloadable]

In [52]:
category = [i.split("/")[1] for i in allfilenames]
stonkfilenames = [i for i in allfilenames if i.split("/")[1] == "day_aggs_v1"]

In [201]:
datestrings = [i.split("/")[-1].split(".")[0] for i in allfilenames]
date = [datetime.date(datetime.strptime(i, "%Y-%m-%d")) for i in datestrings]

In [198]:
current_date = datetime.now()
five_years_ago = current_date - timedelta(days=365*5)
date_five_years_ago = five_years_ago.date()

In [207]:
recent = [i for i,j in zip(stonkfilenames, date) if j > date_five_years_ago]

In [223]:
for i in recent:
    localfilename = "day_aggs/" + i.split("/")[-1]

    try:
        s3.download_file(bucket, i, localfilename)
    except Exception as e:
        print(e)